# Install Packages

In [1]:
%%capture
!pip install -qq scrapy
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-openai
!pip install --upgrade --quiet  rank_bm25
!pip install langchain_experimental
!pip install -U FlagEmbedding

# Data Crawling

In [2]:
import requests
import xml.etree.ElementTree as ET
import json

# URLs of the sitemaps
sitemap_urls = [
    'https://hoatuoimymy.com/product-sitemap1.xml',
    'https://hoatuoimymy.com/product-sitemap2.xml'
]

all_urls = []

# Function to fetch and parse XML
def fetch_sitemap(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        root = ET.fromstring(response.content)
        # Extract all <loc> elements that contain the URLs
        for url_element in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
            all_urls.append(url_element.text)
    except Exception as e:
        print(f"Error fetching or parsing {url}: {e}")

# Fetch and parse both sitemaps
for sitemap_url in sitemap_urls:
    fetch_sitemap(sitemap_url)

# Write the URLs to a JSON file
with open('all_urls.json', 'w') as f:
    json.dump(all_urls, f, indent=4)

print(f"Extracted {len(all_urls)} URLs and saved to all_urls.json")

Extracted 401 URLs and saved to all_urls.json


In [3]:
import pandas as pd
import json
import re
# Specify the filename
filename = '/kaggle/working/all_urls.json'

# Load the all_urls list from the JSON file
with open(filename, 'r') as file:
    all_urls = json.load(file)

print(f"All URLs loaded from {filename}")

All URLs loaded from /kaggle/working/all_urls.json


In [4]:
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup

class CustomSpider(scrapy.Spider):
    name = 'custom_spider'
    start_urls = all_urls

    # Initialize a counter
    request_count = 0

    def parse(self, response):
        self.request_count += 1  # Increment the counter with each request
        description = ""

        # Scraping the review title (h1 tag inside div.product_title)
        review_title = response.css('h1.product-title::text').get()

        if review_title:
            h1_tag = review_title.strip()
        else:
            h1_tag = ""

        # Now, h1_tag contains the content of the h1 tag
        print(h1_tag)


        price = response.css('span.woocommerce-Price-amount')

        if price:
            price = price.get().strip()
        else:
            price = ""
        match = re.search(r'>([\d.,]+₫)<', price)
        if match:
            price = match.group(1)
        # Scraping the ck-content
        ck_contents = response.css('div.woocommerce-Tabs-panel--description')

        for ck_content in ck_contents:
            for element in ck_content.xpath('./*'):
                # Extract the text from h2 and h3 tags
                if element.root.tag == 'h2':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"
                elif element.root.tag == 'h3':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the text from p tags
                elif element.root.tag == 'p':
                    description += ' '.join(element.css('::text').getall()).strip() + "\n"

                # Extract the list items from ul tags
                elif element.root.tag == 'ul':
                    li_tags = element.css('li')
                    for li_tag in li_tags:
                        description += f"- {' '.join(li_tag.css('::text').getall()).strip()}" + "\n"

        # Initialize an empty array to hold image URLs
        image_urls = []

        # Select all the div elements with the specific class
        image_elements = response.css('div.woocommerce-product-gallery__image')

        # Loop through each image element to extract the URLs
        for element in image_elements:
            # Extract the main image URL from the 'data-large_image' attribute
            image_url = element.css('img::attr(data-large_image)').get()

            # Add the extracted image URL to the array
            if image_url:
                image_urls.append(image_url)

        data = {}

        if h1_tag and description:
            data = {
                "url": response.url,  # Add the URL of the request
                "content": description,
                "price": price,
                "title": h1_tag,  # h1_tag is now guaranteed to be a string
                "image_urls": image_urls
            }

            yield data
        # Print out the current request count
        print('====> h1_tag', h1_tag)
        print('====>description', description)
        print('====>image_urls', image_urls)
        print('====>price', price)

        self.logger.info(f"Number of requests done: {self.request_count}")
        self.logger.info(f"Crawled: {response.url}")


# Initialize the Scrapy crawler process
process = CrawlerProcess({
    'LOG_LEVEL': 'INFO',
    'FEEDS': {
        'output.json': {
            'format': 'json',
            'encoding': 'utf8',
            'store_empty': False,
            'fields': None,
            'indent': 4,
        },
    },
    'CLOSESPIDER_TIMEOUT': 60000000000,  # Close the spider after 60 seconds (adjust as needed)
    'DOWNLOAD_DELAY': 3,  # Delay of 2 seconds between each request
})

# Start the spider
process.crawl(CustomSpider)
process.start()

2025-01-24 12:38:14 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-01-24 12:38:14 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.10.0, w3lib 2.2.1, Twisted 24.11.0, Python 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.6.56+-x86_64-with-glibc2.35
2025-01-24 12:38:14 [scrapy.addons] INFO: Enabled addons:
[]
2025-01-24 12:38:14 [scrapy.extensions.telnet] INFO: Telnet Password: 0ae469aa63e0221c
2025-01-24 12:38:14 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.closespider.CloseSpider',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-01-24 12:38:14 [scrapy.crawler] INFO: Overridden settings:
{'CLOSESPIDER_TIMEOUT': 60000000000, 'DOWNLOAD_DELAY': 3, 'LO


====> h1_tag 
====>description 
====>image_urls []
====>price <span class="woocommerce-Price-amount amount"><bdi>650.000<span class="woocommerce-Price-currencySymbol">₫</span></bdi></span>


2025-01-24 12:38:18 [custom_spider] INFO: Number of requests done: 2
2025-01-24 12:38:18 [custom_spider] INFO: Crawled: https://hoatuoimymy.com/gio-hoa-m487/


Giỏ Hoa M487
====> h1_tag Giỏ Hoa M487
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Giỏ Hoa M482
Giỏ Hoa M487 – Món Quà Tươi Sáng và Ý Nghĩa
Giỏ Hoa M487  là sự kết hợp hoàn hảo của những bông hoa tươi đẹp, mang đến vẻ đẹp dịu dàng và ấm áp cho không gian của bạn. Được chăm chút tỉ mỉ từ từng chi tiết, giỏ hoa này không chỉ là món quà tuyệt vời mà còn là biểu tượng của sự tôn trọng, tình yêu thương và lòng biết ơn sâu sắc. Đây là lựa chọn lý tưởng cho nhiều dịp quan trọng trong cuộc sống, từ  sinh nhật ,  kỷ niệ

2025-01-24 12:38:22 [custom_spider] INFO: Number of requests done: 3
2025-01-24 12:38:22 [custom_spider] INFO: Crawled: https://hoatuoimymy.com/gio-hoa-m486/


...

Bó Hoa M98
====> h1_tag Bó Hoa M98
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Bó Hoa M98
Bó Hoa Hồng Sinh Nhật Tại Hoa Tươi My My – Đem Đến Niềm Vui Và Tình Yêu
Giới Thiệu Về Bó Hoa Hồng Sinh Nhật Tại Hoa Tươi My My
Tại  Hoa Tươi My My , chúng tôi cung cấp những bó hoa hồng sinh nhật tuyệt đẹp, hoàn hảo để chúc mừng và thể hiện tình cảm chân thành của bạn. Với thiết kế tinh tế và hoa hồng tươi chất lượng cao, bó hoa hồng sinh nhật của chúng tôi chắc chắn sẽ làm cho ngày sinh nhật của người nhận thêm phần đặc b

2025-01-24 13:02:38 [custom_spider] INFO: Number of requests done: 400
2025-01-24 13:02:38 [custom_spider] INFO: Crawled: https://hoatuoimymy.com/bo-hoa-m97/


Bó Hoa M97
====> h1_tag Bó Hoa M97
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Bó Hoa M97
Bó Hoa Hồng Sinh Nhật Tại Hoa Tươi My My – Đem Đến Niềm Vui Và Tình Yêu
Giới Thiệu Về Bó Hoa Hồng Sinh Nhật Tại Hoa Tươi My My
Tại  Hoa Tươi My My , chúng tôi cung cấp những bó hoa hồng sinh nhật tuyệt đẹp, hoàn hảo để chúc mừng và thể hiện tình cảm chân thành của bạn. Với thiết kế tinh tế và hoa hồng tươi chất lượng cao, bó hoa hồng sinh nhật của chúng tôi chắc chắn sẽ làm cho ngày sinh nhật của người nhận thêm phần đặc b

2025-01-24 13:02:42 [custom_spider] INFO: Number of requests done: 401
2025-01-24 13:02:42 [custom_spider] INFO: Crawled: https://hoatuoimymy.com/bo-hoa-tang-nguoi-yeu/
2025-01-24 13:02:42 [scrapy.core.engine] INFO: Closing spider (finished)
2025-01-24 13:02:42 [scrapy.extensions.feedexport] INFO: Stored json feed (400 items) in: output.json
2025-01-24 13:02:42 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 93996,
 'downloader/request_count': 401,
 'downloader/request_method_count/GET': 401,
 'downloader/response_bytes': 13676486,
 'downloader/response_count': 401,
 'downloader/response_status_count/200': 401,
 'elapsed_time_seconds': 1468.345863,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2025, 1, 24, 13, 2, 42, 568662, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 62211541,
 'httpcompression/response_count': 401,
 'item_scraped_count': 400,
 'items_per_minute': 

Bó Hoa Tặng Người Yêu
====> h1_tag Bó Hoa Tặng Người Yêu
====>description Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí:
- Hoa tươi mới được nhập về trong ngày
- Cập nhật xu hướng hoa mới nhất trên thị trường
- Các thiết kế hoa độc lạ và cực kỳ bắt mắt
- Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận
- Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi
- Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách
- Hoàn trả tiền khi khách hàng không hài lòng
- Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop
Bó Hoa Hồng Tặng Người Yêu
Bó Hoa Hồng Tặng Người Yêu Tại Hoa Tươi My My – Tạo Dấu Ấn Đặc Biệt Trong Mỗi Dịp
Giới Thiệu Bó Hoa Hồng Tặng Người Yêu Tại Hoa Tươi My My
Tại Hoa Tươi My My, chúng tôi hiểu rằng mỗi khoảnh khắc bên người yêu đều đáng trân trọng. Bó hoa hồng của chúng tôi là cách hoàn hảo để thể hiện tình yêu và sự quan tâm sâu sắc đến người bạn yêu thương. Với các thiết kế tinh tế và chất lượng hoa hồ

# Data Processing

In [22]:
import json

# Load the `output.json` file
with open('/kaggle/input/flower-chatbot/output.json', 'r',encoding="utf8") as f:
    data = json.load(f)

In [23]:
len(data)

400

# Remove excess characters and duplicate text

In [24]:
import re
# Remove special characters
for item in data:
  item['content']= re.sub(r'[^\S ]+', ' ', item['content'])

In [25]:
data[0]

{'url': 'https://hoatuoimymy.com/gio-hoa-m487/',
 'content': 'Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí: - Hoa tươi mới được nhập về trong ngày - Cập nhật xu hướng hoa mới nhất trên thị trường - Các thiết kế hoa độc lạ và cực kỳ bắt mắt - Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận - Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi - Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách - Hoàn trả tiền khi khách hàng không hài lòng - Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop Giỏ Hoa M482 Giỏ Hoa M487 – Món Quà Tươi Sáng và Ý Nghĩa Giỏ Hoa M487  là sự kết hợp hoàn hảo của những bông hoa tươi đẹp, mang đến vẻ đẹp dịu dàng và ấm áp cho không gian của bạn. Được chăm chút tỉ mỉ từ từng chi tiết, giỏ hoa này không chỉ là món quà tuyệt vời mà còn là biểu tượng của sự tôn trọng, tình yêu thương và lòng biết ơn sâu sắc. Đây là lựa chọn lý tưởng cho nhiều dịp quan trọng trong cuộc sống, từ  sinh nhật ,  

In [26]:
# The duplicated text of most records
introduction = """Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí: - Hoa tươi mới được nhập về trong ngày - Cập nhật xu hướng hoa mới nhất trên thị trường - Các thiết kế hoa độc lạ và cực kỳ bắt mắt - Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận - Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi - Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách - Hoàn trả tiền khi khách hàng không hài lòng - Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop"""

In [27]:
count = 0
exception = []
for item in data:
  if introduction in item['content']:
    count = count + 1
    item['content'] = item['content'].replace(introduction,"")
  else:
    exception.append(item)
print(count)

400


In [28]:
data[0]['content']

' Giỏ Hoa M482 Giỏ Hoa M487 – Món Quà Tươi Sáng và Ý Nghĩa Giỏ Hoa M487  là sự kết hợp hoàn hảo của những bông hoa tươi đẹp, mang đến vẻ đẹp dịu dàng và ấm áp cho không gian của bạn. Được chăm chút tỉ mỉ từ từng chi tiết, giỏ hoa này không chỉ là món quà tuyệt vời mà còn là biểu tượng của sự tôn trọng, tình yêu thương và lòng biết ơn sâu sắc. Đây là lựa chọn lý tưởng cho nhiều dịp quan trọng trong cuộc sống, từ  sinh nhật ,  kỷ niệm ,  chúc mừng thành công  đến  tân gia  và các dịp đặc biệt khác. - Hoa Tươi Đẹp và Cao Cấp:   Giỏ Hoa M487  được tạo thành từ những bông hoa tươi thắm, có thể bao gồm nhiều loài hoa khác nhau như  hoa hồng ,  hoa đồng tiền ,  hoa ly , hay  lan . Mỗi bông hoa đều được chọn lựa kỹ càng, đảm bảo sự tươi mới và chất lượng, mang lại vẻ đẹp hoàn hảo cho giỏ hoa. - Thiết Kế Sang Trọng và Tinh Tế:  Giỏ hoa được thiết kế với sự kết hợp hoàn hảo giữa các hoa và lá trang trí, tạo nên một tổng thể hài hòa và đẹp mắt.  Giỏ hoa  có thể được đựng trong  giỏ đan ,  hộp hoa

# Convert all the records into LangChain's Document format

In [30]:
from uuid import uuid4
from bs4 import BeautifulSoup
from langchain_core.documents import Document

In [36]:
list_of_documents = []
for item in data:
  # Extract the price value from HTML tag
  soup = BeautifulSoup(item['price'], 'html.parser')
  price = soup.bdi.text.strip()
  content = item['title'] + ' - Giá tiền: ' + price + ' - ' + item['content']
  # Convert the records to LangChain's Document format and append them to a list
  list_of_documents.append(Document(page_content=content, metadata={"source": item['url'],
                                                                    "image_urls":item['image_urls']}))

In [37]:
list_of_documents.append(Document(page_content = introduction))

In [39]:
len(list_of_documents)

401

In [40]:
list_of_documents[-1]

Document(metadata={}, page_content='Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí: - Hoa tươi mới được nhập về trong ngày - Cập nhật xu hướng hoa mới nhất trên thị trường - Các thiết kế hoa độc lạ và cực kỳ bắt mắt - Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận - Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi - Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách - Hoàn trả tiền khi khách hàng không hài lòng - Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop')

# Semantic Chunking

In [75]:
from transformers import AutoTokenizer
from langchain.embeddings import HuggingFaceEmbeddings

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

2025-01-24 13:54:04 [sentence_transformers.SentenceTransformer] INFO: Use pytorch device_name: cuda
2025-01-24 13:54:04 [sentence_transformers.SentenceTransformer] INFO: Load pretrained SentenceTransformer: BAAI/bge-m3


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [76]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(embed_model, breakpoint_threshold_type="percentile", breakpoint_threshold_amount=85)

In [77]:
# Split the documents and count the number of chunks in each document, as well as the number of tokens in each chunk
count = 0
total = 0
list_of_chunks = []
for idx,doc in enumerate(list_of_documents):
  chunks = text_splitter.create_documents([doc.page_content])
  print(f'Number of chunks: {len(chunks)} - Tokens of each chunk',end=' ')
  for chunk in chunks:
      text = chunk.page_content
      tokens = tokenizer.tokenize(text)
      num_tokens = len(tokens)
      if num_tokens > 1:
        total = total + 1
        # Use the parent document index as metadata to retrieve the parent document from the child chunk.
        chunk.metadata['parent'] = idx
        list_of_chunks.append(chunk)
      if num_tokens > 512:
        count = count + 1
      print(num_tokens, end =' ')
  print()
print('Toltal chunks: ',total)
print('Number of chunks which is larger than 512 tokens: ',count)

Number of chunks: 3 - Tokens of each chunk 112 125 345 
Number of chunks: 5 - Tokens of each chunk 220 282 301 97 0 
Number of chunks: 5 - Tokens of each chunk 277 531 52 98 0 
Number of chunks: 5 - Tokens of each chunk 410 389 52 97 0 
Number of chunks: 4 - Tokens of each chunk 102 488 213 0 
Number of chunks: 4 - Tokens of each chunk 105 138 562 0 
Number of chunks: 4 - Tokens of each chunk 102 149 506 0 
Number of chunks: 4 - Tokens of each chunk 248 210 280 110 
Number of chunks: 5 - Tokens of each chunk 248 377 74 52 110 
Number of chunks: 5 - Tokens of each chunk 119 147 322 235 109 
Number of chunks: 5 - Tokens of each chunk 337 273 276 95 0 
Number of chunks: 4 - Tokens of each chunk 491 110 173 0 
Number of chunks: 5 - Tokens of each chunk 122 129 573 105 0 
Number of chunks: 5 - Tokens of each chunk 270 496 52 103 0 
Number of chunks: 5 - Tokens of each chunk 273 83 437 106 0 
Number of chunks: 5 - Tokens of each chunk 250 91 76 374 117 
Number of chunks: 5 - Tokens of each c

In [78]:
len(list_of_chunks)

1239

In [79]:
list_of_chunks[-1]

Document(metadata={'parent': 400}, page_content='Hoa Tươi My My luôn là lựa chọn tốt nhất của những tín đồ yêu thích hoa. Với tiêu chí: - Hoa tươi mới được nhập về trong ngày - Cập nhật xu hướng hoa mới nhất trên thị trường - Các thiết kế hoa độc lạ và cực kỳ bắt mắt - Điện hoa nhanh chóng trong nội thành và các tỉnh lân cận - Hình ảnh hoa được cập nhật trước cho khách hàng khi gửi - Hoa đến tay đảm bảo còn tươi mới, đẹp và đáp ứng được mọi yêu cầu của khách - Hoàn trả tiền khi khách hàng không hài lòng - Bạn có thể đặt hoa nhanh ship 2-3h tại zalo shop')

In [80]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(list_of_chunks))]

# Retriever Module

## FAISS Vector Store

In [85]:
import faiss
from langchain.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
)

2025-01-24 14:02:52 [sentence_transformers.SentenceTransformer] INFO: Use pytorch device_name: cuda
2025-01-24 14:02:52 [sentence_transformers.SentenceTransformer] INFO: Load pretrained SentenceTransformer: BAAI/bge-m3


In [86]:
index = faiss.IndexFlatL2(len(embedding_model.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [87]:
vector_store.add_documents(documents=list_of_chunks, ids=uuids)

['ca9548f6-378f-4edb-84cd-e64694f21d7f',
 '0eb4e983-0596-4f04-bec1-86546f0475ca',
 '0e63dc5f-f862-409e-bfa6-58366ffa3eb6',
 '863351d1-9951-4c61-b1a8-51106c58abf2',
 '9f7d1367-b4c6-423b-b333-cd7299bf18d2',
 '6e6031f7-8b83-4b77-8463-1484edb04f63',
 'd7b81299-6b61-4967-a84d-8d42fd876209',
 '397ed63d-8241-40c0-b0d8-f7c2bda978cb',
 'aab824e7-551a-46e5-ac47-6aa4094a486d',
 '5373f963-7b6e-49a5-a147-d4314cce10e2',
 'a19c55d9-1e5a-4f3d-bbee-abb38362a438',
 '02e92c59-0884-46be-abf7-34f89edd2dee',
 '6388dbef-1bc2-4a1c-ba13-23d3b3b2a187',
 'b44396d3-ba6e-4989-bc0a-9c19e4773e51',
 '4fa5ccc4-846d-4d51-b2e9-49c4d22aaa87',
 'fa3a5546-473b-479a-a075-5f894949b9bc',
 '46400ec8-a46f-4ccc-a516-d5cc15ec7480',
 'e773309d-cde3-4fe5-bbb4-b34400b6c0de',
 'a7d78496-624b-4d05-ab46-52cd62e8380b',
 'ddde3973-bc96-41b0-8e37-b5fdec88fa51',
 '65a3e422-8e8f-4676-8bcf-e413da281ac7',
 '23d339a1-4e29-4bc3-99ff-6e79081548d1',
 '43de2595-9a89-4469-8020-219d3af04f4b',
 'e08c8e04-6674-4fd4-a0ca-0464910aca69',
 '3d2943a3-3f59-

In [98]:
semantic_results = vector_store.similarity_search(
    "Hoa hướng dương",
    k=10,
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* - Hy Vọng và Năng Lượng Tích Cực:   Hoa hướng dương  là biểu tượng của sự lạc quan, hy vọng và sức sống mạnh mẽ. Hình dáng và màu sắc rực rỡ của hoa hướng dương luôn hướng về mặt trời, là lời nhắc nhở về những điều tốt đẹp trong cuộc sống. Cùng với hoa hồng vàng, giỏ hoa này mang đến thông điệp về sự tươi mới, hạnh phúc và những điều may mắn. [{'parent': 27}]
* Điểm nổi bật của bó hoa hướng dương tốt nghiệp : - Hướng dương tươi sáng : Những bông hướng dương vàng rực rỡ trong bó hoa không chỉ làm nổi bật sự kiện tốt nghiệp mà còn mang đến cảm giác vui tươi và đầy năng lượng. - Thiết kế nổi bật : Bó hoa được thiết kế tinh tế với sự kết hợp hoàn hảo giữa hoa hướng dương và các phụ kiện trang trí khác, tạo nên một tổng thể đẹp mắt và ý nghĩa. - Chất lượng đảm bảo : Tại  Shop Hoa Tươi My My , chúng tôi cam kết sử dụng các bông hoa tươi mới nhất, được chọn lọc kỹ lưỡng để đảm bảo bó hoa luôn giữ được độ tươi lâu và vẻ đẹp hoàn hảo. 🎓  Ưu đãi đặc biệt khi mua bó hoa hướng dương tốt nghiệp t

# BM25 Retrieve

In [91]:
from langchain_community.retrievers import BM25Retriever

In [99]:
bm25_retriever = BM25Retriever.from_documents(
  list_of_chunks, k = 10
)

In [100]:
bm25_results = bm25_retriever.invoke("Hoa hướng dương")
bm25_results

[Document(metadata={'parent': 378}, page_content='Bó Hoa Hướng Dương M114 - Giá tiền: 650.000₫ -  Bó Hoa Hướng Dương M114 Bó Hoa Hướng Dương M114 Tại Hoa Tươi My My – Biểu Tượng Của Sự Lạc Quan Và Hy Vọng Giới Thiệu Bó Hoa Hướng Dương M114 Tại Hoa Tươi My My Bó hoa hướng dương M114 tại Hoa Tươi My My mang trong mình thông điệp của sự lạc quan, sức sống mãnh liệt, và niềm tin vào tương lai tươi sáng. Với những bông hướng dương rực rỡ, bó hoa này không chỉ là món quà đầy ý nghĩa mà còn là điểm nhấn nổi bật trong mọi không gian. Điểm Nổi Bật Của Bó Hoa Hướng Dương M114 Tại Hoa Tươi My My Lợi Ích Của Bó Hoa Hướng Dương M114 - Mang Đến Năng Lượng Tích Cực : Hướng dương luôn hướng về phía mặt trời, tượng trưng cho sự lạc quan và niềm tin vào cuộc sống. Bó hoa hướng dương M114 là món quà hoàn hảo để gửi đến người bạn yêu quý, mang lại nguồn năng lượng tích cực và khích lệ họ trong những thời điểm khó khăn. - Phù Hợp Với Nhiều Dịp Đặc Biệt : Bó hoa hướng dương M114 là lựa chọn tuyệt vời cho cá

# BGE-M3 Reranker

In [101]:
# Get the result chunks from BM25 Retriever and FAISS vector search
content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

16

In [103]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [104]:
pairs = [["Hoa hướng dương",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)
score

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[0.9967013487421672,
 0.8609547092195301,
 0.9550683569776217,
 0.9737945865504661,
 0.9684409808294261,
 0.8570517499769504,
 0.8931231151348971,
 0.8883720089593933,
 0.9503188453090927,
 0.9448931005611606,
 0.9561454366001265,
 0.9145963319978375,
 0.6025878991261147,
 0.7331653714055038,
 0.9396923600249307,
 0.909264512360149]

# Retrieve

In [105]:
# Put all the methods above into a class called 'Retriever'
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self,query):
    semantic_results = self.semantic_retriever.similarity_search(
      query,
      k=10,
    )
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    retrieval_docs = []

    for result in semantic_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    for result in bm25_results:
      if result.page_content not in content:
        content.add(result.page_content)
        retrieval_docs.append(result)

    pairs = [[query,doc.page_content] for doc in retrieval_docs]

    scores = self.reranker.compute_score(pairs,normalize = True)

    # Retrieve the parent document from the child chunk based on a threshold score.
    context_1 = []
    context_2 = []
    context = []
    parent_ids = set()
    for i in range(len(retrieval_docs)):
      # Relevance score >= 0.6 will be used as context type 1 (indicating higher relevance to the query)
      if scores[i] >= 0.6:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_1.append(list_of_documents[parent_idx])
      # Relevance score >= 0.1 will be used as context type 2 (indicating medium to lower relevance to the query)
      elif scores[i] >= 0.1:
        parent_idx = retrieval_docs[i].metadata['parent']
        if parent_idx not in parent_ids:
          parent_ids.add(parent_idx)
          context_2.append(list_of_documents[parent_idx])
      # If the relevance score < 0.1, it indicates that there are no relevant documents.
    if len(context_1) > 0:
      print('Context 1')
      context=context_1
    elif len(context_2) > 0:
      print('Context 2')
      context=context_2
    else:
      print('No relevant context')
    return context

In [106]:
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = reranker)
context = retriever("Hoa hướng dương")

Context 1


In [107]:
context

[Document(metadata={'source': 'https://hoatuoimymy.com/gio-hoa-m459/', 'image_urls': ['https://hoatuoimymy.com/wp-content/uploads/2024/11/31247f15-363f-453a-92b3-568465faa6ba.jpeg']}, page_content='Giỏ Hoa M459 - Giá tiền: 750.000₫ -  Giỏ Hoa M459 Giỏ Hoa M459 – Giỏ Hoa Hướng Dương và Hồng Vàng – Món Quà Tươi Mới, Rực Rỡ Và Ý Nghĩa Giỏ Hoa M459  là sự kết hợp hoàn hảo giữa  hoa hướng dương  vàng rực rỡ và  hoa hồng vàng  tươi tắn, tạo nên một món quà đầy năng lượng, vui tươi và tràn đầy ý nghĩa. Đây là lựa chọn tuyệt vời để gửi gắm những lời chúc mừng, sự quan tâm, tình yêu và những hy vọng tốt đẹp đến người nhận. Với vẻ đẹp rực rỡ và tươi mới,  Giỏ Hoa M459  là món quà lý tưởng cho các dịp như sinh nhật, kỷ niệm, mừng thọ, khai trương, hay bất kỳ dịp đặc biệt nào. - Hoa Hướng Dương Tươi Tắn và Hoa Hồng Vàng Tỏa Sáng:   Giỏ Hoa M459  kết hợp giữa  hoa hướng dương  vàng tươi rực rỡ và  hoa hồng vàng  mang lại một cảm giác tràn đầy năng lượng tích cực, sự vui vẻ và hạnh phúc. Hoa hướng d

# Answer Generation

### Sematic Router

In [109]:
flowerSamples = [
    "Tôi muốn mua hoa hồng đỏ để tặng vào ngày kỷ niệm, bạn có mẫu nào đẹp không?",
    "Có loại hoa nào phù hợp để chúc mừng khai trương không?",
    "Tôi cần một lẵng hoa màu vàng để chúc mừng sinh nhật, có loại nào không?",
    "Bạn có hoa cưới dành riêng cho cô dâu không?",
    "Tôi muốn mua hoa để tặng mẹ nhân ngày của Mẹ, có gợi ý gì không?",
    "Hoa nào đẹp và lãng mạn để tặng người yêu vào ngày lễ tình nhân?",
    "Tôi cần hoa để viếng đám tang, có loại nào phù hợp không?",
    "Có bó hoa nào ý nghĩa để chúc mừng tốt nghiệp không?",
    "Bạn có thể giới thiệu một bó hoa ngọt ngào để tặng người yêu không?",
    "Tôi muốn mua hoa để tặng ngày Phụ nữ Việt Nam, bạn có mẫu nào phù hợp không?",
    "Bó hoa nào phù hợp để tri ân thầy cô vào ngày Nhà giáo Việt Nam?",
    "Tôi muốn đặt một giỏ hoa để chúc mừng công ty mới khai trương.",
    "Bạn có hoa lan để làm quà tặng không?",
    "Tôi cần một bó hoa tulip cho ngày kỷ niệm, bạn có loại nào không?",
    "Hoa nào phù hợp để tặng vào dịp Tết Nguyên Đán?",
    "Có bó hoa nào dễ thương cho sinh nhật bé gái không?",
    "Tôi muốn mua giỏ hoa dành tặng bà nhân ngày lễ lớn, có loại nào không?",
    "Bạn có thể làm một bó hoa kiểu cổ điển không?",
    "Tôi cần hoa để trang trí bàn tiệc cưới, bạn có loại nào sang trọng không?",
    "Hoa nào phù hợp để tặng đồng nghiệp trong dịp sinh nhật?",
    "Tôi muốn một bó hoa lãng mạn để tặng vợ nhân kỷ niệm ngày cưới.",
    "Tôi cần hoa tươi để trang trí cho một sự kiện lớn, bạn có loại nào phù hợp không?",
    "Hoa nào phù hợp để gửi tặng người thân trong bệnh viện?",
    "Tôi muốn đặt một lẵng hoa để chúc Tết ông bà, bạn có gợi ý nào không?",
    "Có loại hoa nào phù hợp để tặng sinh nhật bạn mà trông nhẹ nhàng không?",
    "Hoa nào thích hợp để chúc mừng bạn mới sinh con?",
    "Tôi muốn đặt hoa cho lễ kỷ niệm thành lập công ty, có mẫu nào trang trọng không?",
    "Có thể làm một bó hoa theo yêu cầu với màu trắng chủ đạo không?",
    "Tôi muốn mua một bó hoa sen để tặng cho ông bà nhân ngày lễ lớn.",
    "Hoa nào thường dùng để tặng trong dịp lễ Phật Đản?",
    "Tôi cần hoa tặng đồng nghiệp trong lễ kỷ niệm công ty, có loại nào không?",
    "Có bó hoa nào phù hợp để gửi tặng người thân đang đi xa không?",
    "Tôi muốn hoa tươi và đơn giản để tặng cho người bạn thân.",
    "Có loại hoa nào mang ý nghĩa may mắn cho dịp Tết không?",
    "Hoa nào đẹp và ý nghĩa để tặng mẹ vào sinh nhật?",
    "Tôi muốn một bó hoa với tông màu hồng để tặng bạn gái, bạn có gợi ý gì không?",
    "Có loại hoa nào dễ bảo quản để làm quà không?",
    "Tôi muốn đặt hoa cho lễ hội mùa xuân, có loại nào phù hợp không?",
    "Hoa nào thích hợp để tặng vào lễ Phật Đản cho người lớn tuổi?",
    "Tôi cần một giỏ hoa để chúc mừng bạn mở cửa hàng mới.",
    "Có bó hoa nào độc đáo để chúc mừng sinh nhật đồng nghiệp không?",
    "Hoa nào đẹp để tặng vợ nhân dịp ngày Phụ nữ?",
    "Tôi muốn đặt hoa cho buổi họp mặt cuối năm, bạn có gợi ý gì không?",
    "Có loại hoa nào tượng trưng cho sự thành công và phát đạt không?",
    "Tôi cần một bó hoa dễ thương để chúc mừng bạn mới thăng chức.",
    "Hoa nào ý nghĩa để tặng bố nhân dịp sinh nhật?",
    "Tôi muốn hoa phù hợp để gửi lời chia buồn, bạn có loại nào không?",
    "Có hoa nào nhẹ nhàng để chúc mừng sinh nhật bé trai không?",
    "Tôi muốn tặng hoa cho cô giáo nhân ngày 20/11, bạn có mẫu nào không?",
    "Hoa nào thích hợp để gửi tặng dịp giáng sinh?",
    "Tôi cần hoa để trang trí không gian hội nghị, bạn có loại nào sang trọng không?",
    "Hoa nào phù hợp để tặng đối tác trong ngày khai trương công ty?",
    "Tôi muốn hoa lan để trưng bày trong dịp Tết Nguyên Đán.",
    "Hoa nào thích hợp để tặng cho em gái nhân dịp sinh nhật?",
    "Tôi cần một bó hoa mang màu sắc tươi sáng cho ngày lễ Phục sinh.",
    "Có loại hoa nào đặc biệt để tặng ông bà nhân dịp mừng thọ không?",
    "Tôi muốn đặt hoa tặng người yêu nhân dịp sinh nhật, có mẫu nào không?",
    "Hoa nào ý nghĩa để chúc mừng đồng nghiệp thăng chức?",
    "Tôi cần hoa để chúc mừng mẹ nhân ngày lễ Quốc tế Phụ nữ.",
    "Có loại hoa nào phù hợp để gửi tặng bạn cũ nhân dịp gặp lại không?",
    "Tôi muốn một giỏ hoa thanh nhã để tặng người lớn tuổi nhân dịp Tết.",
    "Hoa nào đẹp để gửi lời cám ơn đến sếp?",
    "Tôi muốn hoa lan cho sinh nhật bà ngoại, có mẫu nào không?",
    "Có hoa nào dễ chăm sóc để tặng người thân ở bệnh viện không?",
    "Tôi cần hoa để trang trí cho lễ hội đêm giao thừa, bạn có loại nào không?",
    "Hoa nào thích hợp để tặng cho bố nhân dịp lễ Cha?",
    "Tôi muốn đặt hoa theo yêu cầu với sắc đỏ chủ đạo để tặng vào Valentine.",
    "Có bó hoa nào đặc biệt để tặng bạn thân nhân dịp sinh nhật?",
    "Tôi muốn đặt hoa mừng ngày Nhà giáo, có loại nào trang nhã không?",
    "Hoa nào thích hợp để tặng trong lễ kỷ niệm 10 năm ngày cưới?",
    "Có loại hoa nào ý nghĩa để gửi lời xin lỗi không?",
    "Tôi muốn một bó hoa rực rỡ để tặng bạn gái, có mẫu nào không?",
    "Hoa nào phổ biến để gửi tặng người thân trong ngày lễ Halloween?",
    "Tôi cần hoa tặng đồng nghiệp nữ, có mẫu nào nữ tính không?",
    "Có bó hoa nào dành riêng cho dịp tốt nghiệp không?",
    "Tôi muốn hoa để trang trí phòng khách dịp Tết, có loại nào sang trọng không?",
    "Hoa nào ý nghĩa để tặng người thân nhân dịp năm mới?",
    "Tôi muốn đặt hoa cho lễ mừng thọ ông bà, bạn có gợi ý gì không?",
    "Có loại hoa nào đặc biệt dành cho ngày Valentine không?",
    "Tôi cần hoa để tặng người mới khỏi bệnh, có loại nào không?",
    "Hoa nào đẹp và ý nghĩa để gửi tặng thầy cô nhân ngày Nhà giáo?",
    "Tôi muốn hoa đơn giản để gửi tặng bạn bè vào dịp lễ Quốc tế Lao động.",
    "Có bó hoa nào dễ thương để chúc mừng bạn vừa sinh con gái không?",
    "Hoa nào ý nghĩa để tặng nhân dịp kỷ niệm 20 năm ngày cưới?",
    "Tôi cần hoa tặng đồng nghiệp trong lễ chia tay, bạn có mẫu nào không?",
    "Hoa nào mang thông điệp hy vọng để tặng người đang trải qua khó khăn?",
    "Tôi muốn một bó hoa trắng cho lễ tốt nghiệp, có loại nào không?",
    "Có loại hoa nào thích hợp để chúc mừng bạn chuyển nhà không?",
    "Tôi muốn mua hoa để tặng nhân viên vào dịp cuối năm, bạn có gợi ý không?",
    "Hoa nào đặc biệt để tặng bạn gái nhân kỷ niệm 1 năm quen nhau?",
    "Tôi cần hoa để chúc mừng ngày thành lập công ty, bạn có loại nào không?",
    "Hoa nào phù hợp để tặng dịp lễ Tạ ơn?",
    "Tôi muốn đặt hoa cho lễ giáng sinh, có mẫu nào phù hợp không?",
    "Có loại hoa nào phù hợp để trang trí cho lễ hội mùa hè không?",
    "Tôi muốn một bó hoa dễ bảo quản để tặng bà ngoại.",
    "Hoa nào thường dùng để chúc mừng bạn thăng chức?",
    "Tôi muốn hoa nhẹ nhàng để gửi lời cám ơn mẹ nhân dịp đặc biệt.",
    "Hoa nào đẹp để tặng vợ vào lễ kỷ niệm kết hôn?",
    "Tôi cần hoa tặng người mới sinh em bé, bạn có mẫu nào dễ thương không?",
    "Có hoa nào tượng trưng cho tình bạn để tặng bạn thân không?"
]

In [110]:
chatSamples =  [
    "Chào bạn! Hôm nay thời tiết thế nào nhỉ?",
    "Mùa này có gì thú vị không bạn?",
    "Hello shop",
    "Chàp shop",
    "Chào bạn! Hôm nay bạn thế nào?",
    "Bạn thích làm việc ở đây không?",
    "Bạn có thích mùa hè không?",
    "Bạn có thể kể một câu chuyện vui không?",
    "Bạn có nghe loại nhạc nào không?",
    "Bạn nghĩ gì về việc đọc sách vào buổi sáng?",
    "Bạn thích ăn món gì nhất?",
    "Bạn có kế hoạch gì cho cuối tuần không?",
    "Bạn có thích thể thao không?",
    "Bạn có hay du lịch không?",
    "Bạn thích màu gì nhất?",
    "Bạn có câu châm ngôn sống nào không?",
    "Có nơi nào bạn muốn ghé thăm không?",
    "Bạn thích hoạt động ngoài trời không?",
    "Có quyển sách nào bạn muốn giới thiệu không?",
    "Bạn có lời khuyên gì cho một cuộc sống hạnh phúc không?",
    "Bạn có thói quen nào vào buổi sáng không?",
    "Bạn có thú cưng không?",
    "Bạn thích âm nhạc loại nào?",
    "Bạn thích mùa nào nhất trong năm?",
    "Bạn có hay xem phim không?",
    "Bạn có tin vào may mắn không?",
    "Bạn có đam mê nào khác không?",
    "Có điều gì đặc biệt mà bạn muốn học không?",
    "Bạn có lời khuyên nào để có năng lượng tích cực không?",
    "Bạn thích làm gì vào thời gian rảnh?",
    "Có bài hát nào bạn yêu thích không?",
    "Bạn có thói quen tập thể dục không?",
    "Bạn có thể kể về một ngày thú vị của mình không?",
    "Bạn đã bao giờ tham gia sự kiện nào lớn chưa?",
    "Có bộ phim nào mà bạn không bao giờ quên không?",
    "Bạn có thích làm việc cùng đồng nghiệp không?",
    "Bạn có mơ ước nào chưa thực hiện không?",
    "Bạn nghĩ gì về cuộc sống thành thị?",
    "Bạn đã học được gì khi làm việc ở đây?",
    "Bạn có người thân nào cũng làm nghề này không?",
    "Có món nào bạn hay ăn vào mùa hè không?",
    "Bạn có thích mùa đông không?",
    "Có khi nào bạn gặp khách hàng khó tính không?",
    "Bạn có dự định học thêm gì không?",
    "Có gì đặc biệt bạn muốn chia sẻ hôm nay không?",
    "Bạn thích ngắm hoàng hôn hay bình minh?",
    "Bạn nghĩ sao về phong cách sống tối giản?",
    "Có sở thích nào bạn muốn khám phá không?",
    "Bạn có thích làm việc nhóm không?",
    "Bạn có hay chụp ảnh không?",
    "Bạn có tin vào số mệnh không?",
    "Bạn nghĩ gì về việc sống ở vùng quê?",
    "Có trò chơi nào bạn hay chơi không?",
    "Bạn có lời khuyên nào về sức khỏe không?",
    "Bạn có thích nấu ăn không?",
    "Có địa điểm nào bạn muốn đến thử không?",
    "Bạn có thói quen gì vào buổi tối không?",
    "Có món nào bạn đặc biệt ghét không?",
    "Bạn thích tham gia sự kiện nào nhất?",
    "Có bài hát nào làm bạn vui không?",
    "Bạn có thích sưu tầm gì không?",
    "Bạn có thích trồng cây không?",
    "Bạn có lời khuyên nào về việc giảm căng thẳng không?",
    "Bạn có thích làm việc ở đây lâu dài không?",
    "Có bộ phim nào bạn xem lại nhiều lần không?",
    "Bạn thích làm gì vào cuối tuần?",
    "Bạn có tin vào tâm linh không?",
    "Bạn có thể chia sẻ một câu nói yêu thích không?",
    "Có điều gì thú vị về công việc này không?",
    "Bạn có thường đi xem triển lãm nghệ thuật không?",
    "Bạn có nghĩ mình sẽ chuyển nghề không?",
    "Bạn thích đồ uống gì nhất?",
    "Bạn có thói quen viết nhật ký không?",
    "Bạn nghĩ sao về du lịch một mình?",
    "Có trò nào bạn thích chơi từ nhỏ không?",
    "Bạn có thường gặp khách du lịch không?",
    "Bạn có thể chia sẻ bí quyết thành công không?",
    "Bạn nghĩ sao về việc tự kinh doanh?",
    "Bạn có thích đi phượt không?",
    "Có món ăn nào bạn không thể cưỡng lại không?",
    "Bạn nghĩ gì về công việc tình nguyện?",
    "Bạn có sở thích nào mới không?",
    "Bạn có thích chụp ảnh phong cảnh không?",
    "Bạn có thể kể một bí mật nhỏ không?",
    "Có nơi nào bạn muốn quay lại nhiều lần không?",
    "Bạn có thể chia sẻ điều gì đó vui nhộn không?",
]

In [111]:
len(chatSamples),len(flowerSamples)

(86, 100)

In [112]:
from typing import List
class Route():
    def __init__(
        self,
        name: str = None,
        samples:List = []
    ):

        self.name = name
        self.samples = samples

In [114]:
import numpy as np

class SemanticRouter():
    def __init__(self, routes, embedding = embedding_model):
        self.routes = routes
        self.embedding = embedding_model
        self.routesEmbedding = {}

        for route in self.routes:
            self.routesEmbedding[
                route.name
            ] = self.embedding.embed_documents(route.samples)

    def get_routes(self):
        return self.routes

    def guide(self, query):
        queryEmbedding = self.embedding.embed_query(query)
        queryEmbedding = queryEmbedding / np.linalg.norm(queryEmbedding)
        scores = []

        # Calculate the cosine similarity of the query embedding with the sample embeddings of the router.

        for route in self.routes:
            routesEmbedding = self.routesEmbedding[route.name] / np.linalg.norm(self.routesEmbedding[route.name])
            score = np.mean(np.dot(routesEmbedding, queryEmbedding.T).flatten())
            scores.append((score, route.name))

        scores.sort(reverse=True)
        return scores

In [115]:
flowerRoute = Route(name = 'flower', samples = flowerSamples)
chatRoute = Route(name = 'chat', samples = chatSamples)
semanticRoute = SemanticRouter(routes = [flowerRoute, chatRoute])

In [116]:
testSamples = [
    "Tôi muốn mua hoa hồng đỏ để tặng vào ngày kỷ niệm, bạn có mẫu nào đẹp không?",
    "Có loại hoa nào phù hợp để chúc mừng khai trương không?",
    "Tôi cần một lẵng hoa màu vàng để chúc mừng sinh nhật, có loại nào không?",
    "Shop ơi cho mình hỏi một chút",
    "Hello shop",
    "Xin chào, hôm nay bạn thế nào",
    "Tớ muốn tâm sự với cậu một chút",
    "Hi Shop, mình muốn tư vấn"
]

In [117]:
for sample in testSamples:
  route = semanticRoute.guide(sample)
  print(route)

[(0.0672452463765955, 'flower'), (0.050266662871870094, 'chat')]
[(0.06763107293557763, 'flower'), (0.04764250082434152, 'chat')]
[(0.06653534779551104, 'flower'), (0.048234725489361134, 'chat')]
[(0.05429758202810734, 'chat'), (0.04547823469518431, 'flower')]
[(0.05198942082100339, 'chat'), (0.04406298875613403, 'flower')]
[(0.0609749873242049, 'chat'), (0.04690242883853205, 'flower')]
[(0.05831500588371153, 'chat'), (0.045126873961961744, 'flower')]
[(0.05318923296915863, 'chat'), (0.04502134288370906, 'flower')]


# Reflection

In [119]:
!pip install langchain-together

In [177]:
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_together import ChatTogether
import os

os.environ["TOGETHER_API_KEY"] = "4163394ca50d6a0ff58f7953bd8cec3167d092c7cd3f6bc624e04a2af728344c"

chat_model = ChatTogether(
    temperature=0.5,
    model_name="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    # api_key=userdata.get("GROQ_API_KEY")
    # max_tokens=4000,
)

In [178]:
class Reflection():
    def __init__(self, llm):
        self.llm = llm


    def __call__(self, chatHistory, lastItemsConsidereds=100):

        if len(chatHistory) >= lastItemsConsidereds:
            chatHistory = chatHistory[len(chatHistory) - lastItemsConsidereds:]

        history_string = "\n".join(
            [
                f"{message['role']} : {message['content']}"
                for message in chatHistory[:-1]
            ]
        )

        chat_template = PromptTemplate.from_template(
            """Given a chat history and the latest user question which might reference context in the chat history,
        formulate a standalone question in Vietnamese which can be understood without the chat history.
        Do NOT answer the question, just reformulate it if needed and otherwise return it as is.

        Chat History:
        {history_string}

        Latest User Question:
        {input}"""
        )

        chain = chat_template | self.llm
        response = chain.invoke({"input": chatHistory[-1]["content"], "history_string": history_string})

        return response.content

In [179]:
reflection = Reflection(chat_model)

In [180]:
chatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Tôi cần tư vấn một số lẵng hoa 2 tầng"},
               {"role": "assistant", "content":"Bạn muốn tư vấn lẵng hoa với hoa gì?"},
               {"role": "user", "content": "Hoa hướng dương"}]

In [181]:
history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in chatHistory[:-1]
    ]
)
history_string

'user : Xin chào shop\nassistant : Chào bạn tôi có thể giúp gì cho bạn\nuser : Tôi cần tư vấn một số lẵng hoa 2 tầng\nassistant : Bạn muốn tư vấn lẵng hoa với hoa gì?'

In [182]:
reflection(chatHistory)

2025-01-24 14:59:55 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


'Câu hỏi của bạn là: Bạn muốn tư vấn về một lẵng hoa 2 tầng có chứa hoa hướng dương.'

# LLMs Generations

In [183]:
rag_template="""\
Hãy trở thành chuyên gia tư vấn bán hàng cho một cửa hàng bán lẵng hoa.
Câu hỏi của khách hàng: {query}.
Trả lời câu hỏi dựa vào các thông tin sản phẩm dưới đây: {source_information}.
Câu trả lời cần ngắn gọn, tập trung, và không thêm thông tin không liên quan.
"""

answerPrompt = PromptTemplate.from_template(rag_template)

In [184]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

semantic_rag_chain = (
    {"source_information" : vector_store.as_retriever(search_kwargs={"k" : 4}), "query" : RunnablePassthrough()}
    | answerPrompt
    | chat_model
    | StrOutputParser()
)

In [185]:
semantic_rag_chain.invoke("tôi muốn mua hoa hướng dương cho ngày lễ tốt nghiệp")

2025-01-24 15:00:05 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


'Chào khách hàng! Tôi rất vui được giúp bạn tìm kiếm bó hoa hướng dương tốt nghiệp phù hợp. \n\nShop Hoa Tươi My My có 2 lựa chọn bó hoa hướng dương tốt nghiệp:\n\n- Bó Hoa Hướng Dương M275 (giá 450.000₫): Bó hoa hướng dương tươi sáng, thiết kế tinh tế và chất lượng đảm bảo.\n- Bó Hoa Hướng Dương M274 (giá 650.000₫): Bó hoa hướng dương tốt nghiệp với màu vàng tươi sáng và ấm áp, thể hiện niềm vui và lời chúc tốt đẹp.\n\nBạn có muốn biết thêm thông tin về 2 lựa chọn này không?'

In [146]:
answerChain = answerPrompt | chat_model

# Agent Tools (using for RAG version 2)

In [160]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Optional, Type, List

In [161]:
# Define a BaseModel Retrieve class for LLM tool calling
class Retrieve(BaseModel):
    """
    Searches the knowledge base for answers. The query parameter should contain the contextualized question to search for in the knowledge base.
    """
    query: str = Field(description="should be a search query")

In [227]:
# The toolPrompt version 1
toolPrompt = PromptTemplate.from_template("""
    You are an AI assistant of a flower shop.
    \\nYou do not know anything about the information and operating procedures of the shop,
    must not answer the user directly, instead *** use the tools to get the most accurate information.***
    \\nIf user just wanna say hello, thank you, normal talk or ask for some life advice, general knowledge, you can reply.
    \\n However, in any other situation, you must use the tool \'Retrieve\' without exception.
    \\n Your customers are Vietnamese, so always reply in Vietnamese.
    \\n Here is your chat history: {chat_history}
""")

In [228]:
# The toolPrompt version 2
toolPrompt = PromptTemplate.from_template("""
    You are an AI assistant of a flower shop.
    For any questions regarding flower products, you ***must use the `Retrieve` tool*** to obtain accurate information.
    To use the `Retrieve` tool, take the user's most recent question as well as relevant chat history, extract a clear, concise search query from the question and chat context. Pass this query to the `Retrieve` tool by setting the `query` parameter.
    For all other questions or general interactions, such as greetings, expressions of gratitude, or requests for life advice or general knowledge, you may reply directly.
    \\n Your customers are Vietnamese, so always reply in Vietnamese.
    \\n Here is your chat history: {chat_history}
""")

In [231]:
llm = ChatTogether(
    temperature=0.4,
    model_name="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
)
agent = llm.bind_tools([Retrieve])
agentChain = toolPrompt | agent

In [232]:
# Test the tool calling with a conversation about Flower products
flowerChatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Tôi cần tư vấn một số lẵng hoa 2 tầng"},
               {"role": "assistant", "content":"Bạn muốn tư vấn lẵng hoa với hoa gì?"},
               {"role": "user", "content": "Hoa hồng"}]
flower_history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in flowerChatHistory
    ]
)
flower_history_string

'user : Xin chào shop\nassistant : Chào bạn tôi có thể giúp gì cho bạn\nuser : Tôi cần tư vấn một số lẵng hoa 2 tầng\nassistant : Bạn muốn tư vấn lẵng hoa với hoa gì?\nuser : Hoa hồng'

In [233]:
response = agentChain.invoke({"chat_history":flower_history_string})

2025-01-24 15:55:10 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


In [234]:
response.additional_kwargs['tool_calls']

[{'id': 'call_3kiq8yg1ohndus0hostfghr3',
  'function': {'arguments': '{"query":"lương hoa 2 tầng hương đương hương hóng"}',
   'name': 'Retrieve'},
  'type': 'function',
  'index': 0}]

In [235]:
# Test the tool calling with a normal, chitchat conversation
chatHistory = [{"role": "user", "content": "Xin chào shop"},
               {"role": "assistant", "content": "Chào bạn tôi có thể giúp gì cho bạn"},
               {"role": "user", "content": "Quốc kỳ Việt Nam có bao nhiêu màu?"}]
history_string = "\n".join(
    [
        f"{message['role']} : {message['content']}"
        for message in chatHistory
    ]
)
history_string

'user : Xin chào shop\nassistant : Chào bạn tôi có thể giúp gì cho bạn\nuser : Quốc kỳ Việt Nam có bao nhiêu màu?'

In [236]:
agentChain.invoke({"chat_history":history_string})

2025-01-24 15:55:20 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Quốc kỳ Việt Nam có 2 màu, đó là màu đỏ và màu vàng.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 487, 'total_tokens': 506, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-39927995-eb1b-4fc0-b649-4825c504f554-0', usage_metadata={'input_tokens': 487, 'output_tokens': 19, 'total_tokens': 506, 'input_token_details': {}, 'output_token_details': {}})

# API

In [199]:
!pip install pyngrok
!pip install flask_cors

In [201]:
import os
import json
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [239]:
ngrok.set_auth_token('2pvM0Nls5lcjnavNxQpqgcdmeF3_uShwFVnbk43vXLQPypyr')

app = Flask(__name__)
CORS(app)  # Apply CORS to the Flask app

2025-01-24 15:55:51 [pyngrok.process] INFO: Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok


In [240]:
@app.route('/v1/chat', methods=['POST'])
def chat_v1():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    context = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Context: {context}')

    query = reflection(context)
    print(f'REFINED: {query}')
    route = semanticRoute.guide(query)[0][1]

    if route == 'flower':
      print('Guide to FLOWER')
      # refine_query = reflection(chatHistory)
      context = retriever(query)
      source_information = ""
      for doc in context:
        content = doc.page_content + ' - Link ảnh: #' + str(doc.metadata['image_urls'])
        source_information+= content + "\n"
      if stream:
        def generate():
          for chunk in answerChain.stream({"query": query, "source_information": source_information}):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerChain.invoke({"query": query, "source_information": source_information})
        return jsonify({'response': reponse.content})
    else:
      print('Guide to CHAT')
      if stream:
        def generate():
          for chunk in answerModel.stream(query):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerModel.invoke(query)
        return jsonify({'response': reponse.content})

@app.route('/v2/chat', methods=['POST'])
def chat_v2():
    # Extract parameters from the request
    user_message = request.json.get('message', {})
    context = request.json.get('context', [])
    stream = True  # Default to False if not provided

    print(f'Message: {user_message}')
    print(f'Context: {context}')

    history_string = "\n".join(
      [
          f"{message['role']} : {message['content']}"
          for message in context
      ]
    )
    agent_response = agentChain.invoke({"chat_history":history_string})
    if 'tool_calls' in agent_response.additional_kwargs:
      print('Guide to FLOWER')
      refine_query = agent_response.tool_calls[0]['args']['query']
      print(f'REFINED: {refine_query}')
      context = retriever(refine_query)
      source_information = ""
      for doc in context:
        content = doc.page_content + ' - Link ảnh: #' + str(doc.metadata['image_urls'])
        source_information+= content + "\n"

      if stream:
        def generate():
          for chunk in answerChain.stream({"query": refine_query, "source_information": source_information}):
            yield chunk.content
        return app.response_class(generate(), mimetype='text/plain')
      else:
        reponse = answerChain.invoke({"query": refine_query, "source_information": source_information})
        return jsonify({'response': reponse.content})
    else:
      print('Guide to CHAT')
      if stream:
        def generate():
          for chunk in agent_response.content.split(" "):
            yield chunk + " "
        return app.response_class(generate(), mimetype='text/plain')
      else:
        return jsonify({'response': agent_response.content})

if __name__ == '__main__':
    # Start ngrok to tunnel the Flask app
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {url}")

    # Start Flask app
    app.run(port=5000)

2025-01-24 15:55:54 [pyngrok.ngrok] INFO: Opening tunnel named: http-5000-0585d153-17f3-4ab2-a975-2298db0e3ac4
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="no configuration paths supplied"
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="client session established" obj=tunnels.session
2025-01-24 15:55:54 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:55:54+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2025-01-24 15

 * ngrok tunnel: NgrokTunnel: "https://a2bb-34-147-71-175.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


2025-01-24 15:55:54 [werkzeug] INFO: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-01-24 15:55:54 [werkzeug] INFO: Press CTRL+C to quit
2025-01-24 15:56:49 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:56:48+0000 lvl=info msg="join connections" obj=join id=5585cb1b7b8b l=127.0.0.1:5000 r=[2401:d800:ce7:b91f:dc22:fd6b:9a8f:a3ef]:65352


Message: {'human': 'xin chào'}
Context: [{'role': 'user', 'content': 'xin chào'}]


2025-01-24 15:56:49 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-24 15:56:49 [werkzeug] INFO: 127.0.0.1 - - [24/Jan/2025 15:56:49] "POST /v2/chat HTTP/1.1" 200 -


...

2025-01-24 15:57:09 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:57:09+0000 lvl=info msg="join connections" obj=join id=7d31f1f66621 l=127.0.0.1:5000 r=[2401:d800:ce7:b91f:dc22:fd6b:9a8f:a3ef]:65372


Message: {'human': 'Tôi muốn bạn tư vấn cho tôi về hoa để tặng bạn thân trong ngày lễ tốt nghiệp'}
Context: [{'role': 'user', 'content': 'xin chào'}, {'role': 'assistant', 'content': 'Xin chào! Tôi có thể giúp gì cho bạn hôm nay? '}, {'role': 'user', 'content': 'Tôi muốn bạn tư vấn cho tôi về hoa để tặng bạn thân trong ngày lễ tốt nghiệp'}]


2025-01-24 15:57:10 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


Guide to FLOWER
REFINED: hoa để tặng bạn thân trong ngày lễ tốt nghiệp
Context 1


2025-01-24 15:57:12 [httpx] INFO: HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-24 15:57:12 [werkzeug] INFO: 127.0.0.1 - - [24/Jan/2025 15:57:12] "POST /v2/chat HTTP/1.1" 200 -
2025-01-24 15:59:40 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:59:40+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
2025-01-24 15:59:40 [pyngrok.process.ngrok] INFO: t=2025-01-24T15:59:40+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
